# Review Analysis Dashboard

## Value and Use of the Dashboard in Back Market Customer Review Analysis

A dashboard helps synthesize and visualize customer review trends to understand user satisfaction, identify recurring issues, and assess the impact of corrective actions. In this context, it provides several benefits:

- **Tracking customer satisfaction**: By visualizing rating and sentiment trends, any decline in perceived quality can be detected early.
- **Identifying frequent issues**: Review analysis helps pinpoint common complaints (e.g., delivery, product quality, customer service).
- **Improving customer service responsiveness**: Measuring response speed and frequency allows Back Market to optimize complaint management.
- **Monitoring temporal trends**: Analyzing reviews by months, days, and hours helps determine whether certain events impact satisfaction.
- **Assessing corrective actions**: If improvements are implemented, tracking review trends can confirm whether they are effective.

---

## Top 10 Key Figures to Include in the Dashboard and Their Importance

1. **Distribution of Ratings (Histogram)**  
   → *Why it’s useful*: Provides a clear view of the proportion of positive and negative reviews. A high percentage of low ratings indicates potential issues.

2. **Average Rating Evolution Over Time (Line Chart)**  
   → *Why it’s useful*: Tracks satisfaction trends over time and evaluates the impact of corrective measures.

3. **Sentiment Distribution of Reviews (Positive, Neutral, Negative - NLP-Based Analysis)**  
   → *Why it’s useful*: Offers a more detailed understanding beyond numerical ratings by identifying emotions expressed in reviews.

4. **Top 10 Keywords or Common Topics in Negative Reviews (Word Cloud or Bar Chart)**  
   → *Why it’s useful*: Identifies the most frequently mentioned issues (e.g., "slow delivery," "defective product").

5. **Response Rate to Reviews (Including Average Response Time)**  
   → *Why it’s useful*: Measures customer service responsiveness and engagement in handling customer feedback.

6. **Proportion of Reviews with vs. Without a Response**  
   → *Why it’s useful*: Highlights whether Back Market responds adequately to customer feedback.

7. **Review Distribution by Day of the Week and Hour (Heatmap)**  
   → *Why it’s useful*: Helps identify peak times for customer reviews, informing customer support staffing strategies.

8. **Correlation Between Rating and Experience Date (Line Chart)**  
   → *Why it’s useful*: Assesses whether ratings are influenced by the purchase period, helping adjust marketing and service strategies.

9. **Comparison of Reviews with vs. Without a Response (Average Ratings)**  
   → *Why it’s useful*: Determines if reviews receiving a response have improved ratings after customer service interaction.

10. **Monthly Review Volume Evolution**  
    → *Why it’s useful*: Measures customer engagement in leaving reviews and identifies periods with the highest review activity.

---

These insights enable Back Market to make data-driven decisions, enhance customer experience, and optimize service strategies.

### Required packages

In [1]:
import dash
from dash import dcc, html, Input, Output, Dash
import pandas as pd
import plotly.express as px
from collections import Counter
import nltk
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import io
import base64